In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time
import requests
import logging
import re
import os
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler(f"amazon_job_scraper_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"),
                        logging.StreamHandler()
                    ])
logger = logging.getLogger(__name__)

# Function to scroll and load all jobs with improved logic
def scroll_to_load_all(driver, max_scrolls=30, wait_time=2):
    """
    Scroll the page to load all content with a maximum number of scrolls
    For Amazon's job site, which loads content dynamically
    """
    scrolls = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    last_job_count = 0
    consecutive_no_change = 0
    
    logger.info("Starting to scroll to load all content...")
    
    while scrolls < max_scrolls:
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait_time)  # Wait time for content to load
        
        # Take screenshot for debugging
        driver.save_screenshot(f"screenshots/amazon_scroll_{scrolls+1}.png")
        
        # Try to find the "Load More" or similar buttons and click them
        try:
            load_more_buttons = driver.find_elements(By.XPATH, 
                "//button[contains(text(), 'Load More') or contains(text(), 'Show More') or contains(@aria-label, 'Load') or contains(@class, 'load-more')]")
            if load_more_buttons:
                for button in load_more_buttons:
                    if button.is_displayed() and button.is_enabled():
                        driver.execute_script("arguments[0].click();", button)
                        logger.info("Clicked 'Load More' button")
                        time.sleep(wait_time + 1)  # Extra wait for new content
        except Exception as e:
            logger.info(f"No 'Load More' button found or error clicking it: {e}")
        
        # Check height and job count to determine if we've loaded all content
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # Try different job card selectors to get an accurate count
        job_selectors = [
            ".job-tile", 
            ".job-listing",
            ".job-card",
            "[data-job-id]"
        ]
        
        current_job_count = 0
        for selector in job_selectors:
            count = len(driver.find_elements(By.CSS_SELECTOR, selector))
            if count > current_job_count:
                current_job_count = count
        
        logger.info(f"Scroll {scrolls+1}: Height {last_height} → {new_height}, Jobs found: {current_job_count}")
        
        # If no change in height and job count, we might have reached the end
        if new_height == last_height and current_job_count == last_job_count:
            consecutive_no_change += 1
            logger.info(f"No change detected ({consecutive_no_change}/3)")
            if consecutive_no_change >= 3:  # If no change for 3 consecutive scrolls
                logger.info("No more content loading after multiple scrolls. Stopping scroll operation.")
                break
        else:
            consecutive_no_change = 0
            
        last_height = new_height
        last_job_count = current_job_count
        scrolls += 1
    
    logger.info(f"Completed scrolling after {scrolls} scrolls. Found approximately {last_job_count} job items.")
    return last_job_count

# Function to handle pagination for Amazon
def handle_pagination(driver, max_pages=20):
    """
    Handle pagination by clicking through all available pages
    """
    page = 1
    all_jobs = []
    
    logger.info("Starting pagination handling...")
    
    while page <= max_pages:
        logger.info(f"Processing page {page}")
        
        # Take screenshot for debugging
        driver.save_screenshot(f"screenshots/amazon_page_{page}.png")
        
        # Wait for job listings to be visible 
        wait_for_job_listings(driver)
        
        # Extract current page's jobs
        jobs_on_page = extract_job_listings_amazon(driver)
        all_jobs.extend(jobs_on_page)
        logger.info(f"Found {len(jobs_on_page)} jobs on page {page}")
        
        # Look for next page button - try multiple selectors
        next_selectors = [
            "a.next", 
            "[data-action='pagination-next']",
            "button.next",
            "//a[contains(@class, 'next')]",
            "//a[contains(text(), 'Next')]",
            "//button[contains(text(), 'Next')]",
            "//span[contains(text(), 'Next')]/parent::a",
            "//span[contains(text(), 'Next')]/parent::button"
        ]
        
        next_button = None
        for selector in next_selectors:
            try:
                if selector.startswith("//"):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                
                for elem in elements:
                    if elem.is_displayed() and not (elem.get_attribute("disabled") or "disabled" in elem.get_attribute("class") or "inactive" in elem.get_attribute("class")):
                        next_button = elem
                        break
                        
                if next_button:
                    break
            except Exception:
                continue
        
        if not next_button:
            logger.info("No next page button found. Reached last page.")
            break
            
        # Check if button is disabled (end of pages)
        if next_button.get_attribute("disabled") or "disabled" in next_button.get_attribute("class"):
            logger.info("Reached last page - Next button is disabled")
            break
            
        # Click next page using JavaScript to avoid intercept issues
        try:
            driver.execute_script("arguments[0].click();", next_button)
            logger.info("Clicked next page button")
            time.sleep(5)  # Wait for page to load
            page += 1
            
            # Wait for job listings to reload
            wait_for_job_listings(driver)
            
            # Take screenshot after page change
            driver.save_screenshot(f"screenshots/amazon_page_{page}_loaded.png")
            
        except Exception as e:
            logger.error(f"Error clicking next page: {e}")
            # Try one more time with a different approach
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(1)
                next_button.click()
                logger.info("Clicked next page button using alternative method")
                time.sleep(5)
                page += 1
            except Exception as e2:
                logger.error(f"Still failed to click next page: {e2}")
                break
            
    return all_jobs

# Helper function to wait for job listings to appear
def wait_for_job_listings(driver, timeout=15):
    """Wait for job listings to appear on the page using multiple possible selectors"""
    selectors = [
        ".job-tile",
        ".job-listing",
        ".job-card",
        "[data-job-id]",
        ".job-title"
    ]
    
    for selector in selectors:
        try:
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            logger.info(f"Job listings found with selector: {selector}")
            return True
        except TimeoutException:
            continue
    
    logger.warning("Could not detect job listings with any known selector")
    return False

# Extract job information from Amazon page
def extract_job_listings_amazon(driver):
    """
    Extract job listings from Amazon jobs page
    Handles different potential Amazon UI structures
    """
    jobs_data = []
    
    # Try different selectors for job elements 
    selectors = [
        {"container": ".job-tile", "type": "tile"},
        {"container": ".job-listing", "type": "listing"},
        {"container": ".job-card", "type": "card"},
        {"container": "[data-job-id]", "type": "data-id"}
    ]
    
    job_elements = []
    used_selector = None
    
    # Try each selector to find job elements
    for selector in selectors:
        try:
            elements = driver.find_elements(By.CSS_SELECTOR, selector["container"])
            if elements and len(elements) > 0:
                job_elements = elements
                used_selector = selector
                logger.info(f"Found {len(elements)} job elements using selector: {selector['container']}")
                
                # Take a screenshot of the found elements (for debugging)
                if len(elements) > 0:
                    try:
                        driver.execute_script("arguments[0].style.border='3px solid red'", elements[0])
                        driver.save_screenshot("screenshots/amazon_job_elements_found.png")
                        driver.execute_script("arguments[0].style.border=''", elements[0])
                    except:
                        pass
                break
        except Exception as e:
            logger.debug(f"Selector {selector['container']} failed: {e}")
            continue
    
    if not job_elements:
        logger.warning("Could not find job elements with any selector. Taking screenshot for debugging.")
        driver.save_screenshot("screenshots/amazon_no_job_elements.png")
        return jobs_data
    
    # Extract detailed information for each job based on the selector type
    for index, job in enumerate(job_elements):
        try:
            # Use different extraction strategies based on container type
            if used_selector["type"] in ["tile", "listing", "card"]:
                # For standard job tiles
                title_elem = job.find_element(By.CSS_SELECTOR, ".job-title, h2, h3, [data-job-title]")
                title = title_elem.text.strip()
                
                # Get link - try different approaches
                link = None
                link_elements = job.find_elements(By.TAG_NAME, "a")
                if link_elements:
                    for link_elem in link_elements:
                        href = link_elem.get_attribute("href")
                        if href and ("/jobs/" in href or "/job/" in href):
                            link = href
                            break
                
                # If still no link, try to find it differently
                if not link:
                    try:
                        # Try to find link in parent element
                        parent = job
                        for _ in range(3):  # Try up to 3 levels up
                            parent = parent.find_element(By.XPATH, "..")
                            link_elems = parent.find_elements(By.TAG_NAME, "a")
                            for link_elem in link_elems:
                                href = link_elem.get_attribute("href")
                                if href and ("/jobs/" in href or "/job/" in href):
                                    link = href
                                    break
                            if link:
                                break
                    except:
                        pass
                
                # Get location
                location = "Not specified"
                location_selectors = [
                    ".location-and-id", 
                    ".location", 
                    "[data-job-location]",
                    ".job-meta"
                ]
                
                for loc_selector in location_selectors:
                    try:
                        loc_elem = job.find_element(By.CSS_SELECTOR, loc_selector)
                        location_text = loc_elem.text.strip()
                        if location_text:
                            location = location_text
                            break
                    except:
                        continue
                
            elif used_selector["type"] == "data-id":
                # For jobs with data-job-id attribute
                title = "Unknown Title"
                link = None
                location = "Not specified"
                
                try:
                    # Try to find title
                    title_elements = job.find_elements(By.CSS_SELECTOR, 
                        "h2, h3, .title, [data-job-title], a")
                    for elem in title_elements:
                        text = elem.text.strip()
                        if text and len(text) > 3:  # Make sure it's substantial text
                            title = text
                            # If it's an anchor tag, also get the link
                            if elem.tag_name == "a":
                                link = elem.get_attribute("href")
                            break
                    
                    # If still no link, try to find separately
                    if not link:
                        link_elements = job.find_elements(By.TAG_NAME, "a")
                        for link_elem in link_elements:
                            href = link_elem.get_attribute("href")
                            if href and ("/jobs/" in href or "/job/" in href):
                                link = href
                                break
                    
                    # Try to find location
                    location_elements = job.find_elements(By.CSS_SELECTOR, 
                        ".location, [data-location], span, div")
                    for elem in location_elements:
                        text = elem.text.strip()
                        if text and ("," in text or "Remote" in text):
                            location = text
                            break
                except Exception as e:
                    logger.debug(f"Error extracting data-id job info: {e}")
            
            # Use job ID as fallback if no link found
            if not link:
                job_id = job.get_attribute("data-job-id")
                if job_id:
                    link = f"https://www.amazon.jobs/en/jobs/{job_id}"
                    logger.info(f"Created link from job ID: {link}")
            
            # Add the extracted job if we have at least a title and link
            if title and title.strip() and link and link.strip():
                # Check for duplicate before adding
                is_duplicate = False
                for existing_job in jobs_data:
                    if existing_job["Title"] == title and existing_job["Link"] == link:
                        is_duplicate = True
                        break
                
                if not is_duplicate:
                    jobs_data.append({
                        "Title": title,
                        "Location": location,
                        "Link": link,
                        "Description": "",  # We'll get descriptions in a separate step
                        "Company": "Amazon"
                    })
                    logger.info(f"Added job: {title} at {location}")
            
        except (StaleElementReferenceException, Exception) as e:
            logger.error(f"Error extracting job details for job {index+1}: {e}")
            continue
    
    return jobs_data

# Function to get job descriptions
def get_job_descriptions(driver, jobs_data, max_descriptions=100):
    """
    Get job descriptions for a batch of jobs by visiting their individual pages.
    Save full page screenshots of each job description in a separate folder.
    """
    logger.info(f"Getting descriptions for {len(jobs_data)} jobs (up to {max_descriptions})")
    
    # Create a dedicated folder for job description screenshots
    job_desc_folder = 'amazon_job_description_screenshots'
    os.makedirs(job_desc_folder, exist_ok=True)
    
    # Store current URL to return to afterward
    original_url = driver.current_url
    original_window = driver.current_window_handle
    
    # Process up to max_descriptions
    for i, job in enumerate(jobs_data[:max_descriptions]):
        if not job.get("Link"):
            continue
            
        logger.info(f"Getting description for job {i+1}/{min(len(jobs_data), max_descriptions)}: {job['Title']}")
        
        # Create a clean filename from the job title
        clean_title = re.sub(r'[\\/*?:"<>|]', "", job['Title'])
        clean_title = re.sub(r'\s+', "_", clean_title)
        clean_title = clean_title[:100] if len(clean_title) > 100 else clean_title
        
        # Create a new tab for each job
        try:
            # Open new tab
            driver.execute_script("window.open('about:blank', '_blank');")
            driver.switch_to.window(driver.window_handles[-1])
            
            # Navigate to job details page
            driver.get(job["Link"])
            time.sleep(5)  # Wait for page to load
            
            # Take a full page screenshot
            # First, get the height of the entire page
            total_height = driver.execute_script("return document.body.scrollHeight")
            
            # Set window size to capture entire page
            driver.set_window_size(1920, total_height)
            
            # Take screenshot and save in the dedicated folder
            screenshot_file = f"{job_desc_folder}/amazon_job_{i+1}_{clean_title}.png"
            driver.save_screenshot(screenshot_file)
            logger.info(f"Saved full-page screenshot to {screenshot_file}")
            
            # Extract description using multiple potential selectors
            description_selectors = [
                "#job-detail",
                ".job-description",
                "#description",
                ".description",
                ".details-container",
                "#job-requirements", 
                "[data-job-description]",
                "article"
            ]
            
            description = ""
            for selector in description_selectors:
                try:
                    desc_elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    if desc_elements:
                        description = desc_elements[0].text.strip()
                        if description:
                            logger.info(f"Got description for '{job['Title']}' ({len(description)} chars)")
                            break
                except Exception as e:
                    logger.debug(f"Selector {selector} failed: {e}")
            
            # Update the job with the description
            if description:
                job["Description"] = description
            
            # Close tab
            driver.close()
            driver.switch_to.window(original_window)
            
        except Exception as e:
            logger.error(f"Error getting description for {job['Title']}: {e}")
            # Make sure we're back to the original window
            try:
                driver.close()
                driver.switch_to.window(original_window)
            except:
                pass
    
    # Return to original page
    try:
        driver.get(original_url)
        time.sleep(3)
    except:
        pass
        
    logger.info(f"Completed fetching descriptions for {min(len(jobs_data), max_descriptions)} jobs")
    return jobs_data

# Function to handle different types of popups
def handle_popups(driver):
    try:
        # Common buttons for accepting cookies, terms, etc.
        popup_selectors = [
            "//button[contains(text(), 'Accept')]", 
            "//button[contains(text(), 'I agree')]",
            "//button[contains(@id, 'accept')]",
            "//button[contains(@class, 'accept')]",
            "//button[contains(text(), 'Continue')]",
            "//button[contains(text(), 'Got it')]",
            "//button[contains(text(), 'Close')]",
            "//button[@aria-label='Close']",
            "//div[contains(@class, 'cookie')]//button",
            "//div[contains(@id, 'consent')]//button"
        ]
        
        for xpath in popup_selectors:
            try:
                buttons = driver.find_elements(By.XPATH, xpath)
                for button in buttons:
                    if button.is_displayed():
                        button.click()
                        logger.info(f"Clicked popup/cookie button with xpath: {xpath}")
                        time.sleep(1)
            except Exception:
                continue
                
        # Handle alerts
        try:
            alert = Alert(driver)
            alert.accept()
            logger.info("Accepted alert popup")
        except:
            pass
            
    except Exception as e:
        logger.warning(f"Error handling popups: {e}")

# Function to validate URL
def is_valid_link(url):
    if not url or not isinstance(url, str) or not url.startswith("http"):
        return False
    
    # For Amazon links, we'll assume they're valid without checking
    if "amazon.jobs" in url:
        return True
        
    try:
        response = requests.head(url, timeout=5, allow_redirects=True)
        return response.status_code < 400  # Accept any non-error status
    except requests.RequestException:
        logger.warning(f"Invalid link: {url}")
        return False

# Main Amazon job scraper function
def scrape_amazon_jobs(search_keyword="", max_pages=20, headless=False):
    """
    Scrape Amazon jobs with comprehensive approach including pagination.
    
    Parameters:
    search_keyword (str): Keyword to search for (empty string for all jobs)
    max_pages (int): Maximum number of pages to scrape
    headless (bool): Whether to run in headless mode
    
    Returns:
    list: List of all job dictionaries found
    """
    options = webdriver.ChromeOptions()
    
    if headless:
        options.add_argument('--headless')
        
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36')
    
    # Create directories for debugging
    os.makedirs('screenshots', exist_ok=True)
    
    driver = None
    jobs_data = []

    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.set_page_load_timeout(30)
        
        # Construct search URL
        search_term = search_keyword.replace(" ", "+") if search_keyword else ""
        if search_term:
            base_url = f"https://www.amazon.jobs/en/search?base_query={search_term}"
        else:
            base_url = "https://www.amazon.jobs/en/"

        # Open the Amazon careers page
        logger.info(f"Scraping jobs from Amazon" + (f", searching for '{search_keyword}'" if search_keyword else ""))
        driver.get(base_url)
        driver.save_screenshot("screenshots/amazon_initial.png")
        
        # Handle popups
        handle_popups(driver)
        
        # Wait for results to load
        if not wait_for_job_listings(driver, timeout=20):
            logger.warning("Could not detect job search results loading. Trying manual search...")
            
            # Try to search directly by submitting a search form
            try:
                search_box = driver.find_element(By.CSS_SELECTOR, "input[type='search'], input[name='q'], input[name='base_query']")
                search_box.clear()
                search_box.send_keys(search_keyword)
                search_box.send_keys(Keys.RETURN)
                time.sleep(5)
                logger.info("Tried manual search submission")
                driver.save_screenshot("screenshots/amazon_manual_search.png")
                
                # Wait again for results
                wait_for_job_listings(driver, timeout=15)
            except Exception as e:
                logger.error(f"Manual search also failed: {e}")
        
        # Scroll to load all results on first page
        job_count = scroll_to_load_all(driver, max_scrolls=20, wait_time=3)
        driver.save_screenshot("screenshots/amazon_after_scroll.png")
        
        # Handle pagination and collect all jobs
        all_jobs = handle_pagination(driver, max_pages=max_pages)
        logger.info(f"Collected {len(all_jobs)} total jobs after pagination")
        
        # Get job descriptions for all collected jobs
        jobs_with_descriptions = get_job_descriptions(driver, all_jobs, max_descriptions=200)
        
        # Save all jobs regardless of search keyword
        jobs_data = jobs_with_descriptions
        logger.info(f"Saving all {len(jobs_data)} jobs found in search results")

    except Exception as e:
        logger.error(f"Error during scraping from Amazon: {e}")
        if driver:
            driver.save_screenshot("screenshots/amazon_error.png")

    finally:
        if driver:
            driver.quit()

    return jobs_data

# Main function to scrape and save results to CSV
def main(search_keyword="", max_pages=20, headless=False):
    start_time = time.time()
    logger.info(f"Starting Amazon job scraper at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Create screenshots directory
    os.makedirs('screenshots', exist_ok=True)
    
    # Create job description screenshots directory
    os.makedirs('amazon_job_description_screenshots', exist_ok=True)
    
    # Scrape Amazon jobs
    jobs_data = scrape_amazon_jobs(search_keyword, max_pages, headless)
    
    # Generate filenames with timestamps
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    keyword_slug = search_keyword.replace(' ', '_') if search_keyword else 'all'
    detailed_filename = f"amazon_jobs_detailed_{keyword_slug}_{timestamp}.csv"
    simple_filename = f"amazon_jobs_simple_{keyword_slug}_{timestamp}.csv"
    
    # Save results
    if jobs_data:
        # Create DataFrame
        df = pd.DataFrame(jobs_data)
        
        # Save detailed CSV with descriptions
        df.to_csv(detailed_filename, index=False, encoding='utf-8-sig')
        logger.info(f"Detailed jobs saved to '{detailed_filename}'")
        
        # Create and save a simplified CSV without descriptions
        simple_df = df[['Title', 'Location', 'Link', 'Company']].copy()
        simple_df.to_csv(simple_filename, index=False, encoding='utf-8-sig')
        logger.info(f"Simplified jobs list saved to '{simple_filename}'")
        
        # Also create a filtered file if a search keyword was provided
        if search_keyword:
            filtered_df = df[df['Title'].str.contains(search_keyword, case=False)].copy()
            filtered_filename = f"amazon_jobs_filtered_{keyword_slug}_{timestamp}.csv"
            filtered_df.to_csv(filtered_filename, index=False, encoding='utf-8-sig')
            logger.info(f"Jobs filtered by keyword '{search_keyword}' saved to '{filtered_filename}'")
            logger.info(f"Found {len(filtered_df)} jobs matching the keyword out of {len(df)} total jobs")
        
        # Print summary
        print("\n" + "="*50)
        print(f"Amazon Job Scraping Results:")
        print(f"Total jobs found: {len(df)}")
        print(f"Unique locations: {len(df['Location'].unique())}")
        print(f"Sample jobs:")
        print(df[['Title', 'Location']].head())
        print("\nTop locations:")
        print(df['Location'].value_counts().head())
        print(f"\nResults saved to:")
        print(f"- {detailed_filename}")
        print(f"- {simple_filename}")
        if search_keyword:
            print(f"- {filtered_filename}")
        print("="*50)
        
        elapsed_time = time.time() - start_time
        logger.info(f"Completed in {elapsed_time:.2f} seconds")
        
        return df
    else:
        logger.warning(f"No jobs found with search keyword '{search_keyword}'")
        print("\nNo jobs found to display.")
        
        elapsed_time = time.time() - start_time
        logger.info(f"Process completed with no results in {elapsed_time:.2f} seconds")
        
        return pd.DataFrame()

if __name__ == "__main__":
    print("Amazon Job Scraper")
    print("="*50)
    
    # Get user input
    job_title = input("Enter job title to search for (leave empty to get all jobs): ").strip()
    
    try:
        max_pages = int(input("Maximum number of pages to scrape (default 20): ") or "20")
    except ValueError:
        max_pages = 20
        print("Invalid input. Using default of 20 pages.")
    
    headless_mode = input("Run in headless mode? (y/n, default: n): ").strip().lower() == 'y'
    
    print("\nStarting job scraper...")
    print("This may take several minutes depending on the number of jobs and pages.")
    print("Progress will be logged to the console and a log file.")
    
    # Run the scraper
    main(job_title, max_pages, headless_mode)

Amazon Job Scraper


Enter job title to search for (leave empty to get all jobs):  
Maximum number of pages to scrape (default 20):  1
Run in headless mode? (y/n, default: n):  y


2025-03-18 18:35:34,840 - INFO - Starting Amazon job scraper at 2025-03-18 18:35:34
2025-03-18 18:35:34,846 - INFO - ====== WebDriver manager ======



Starting job scraper...
This may take several minutes depending on the number of jobs and pages.
Progress will be logged to the console and a log file.


2025-03-18 18:35:35,170 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-18 18:35:35,262 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-18 18:35:35,327 - INFO - Driver [/Users/srikar/.wdm/drivers/chromedriver/mac64/134.0.6998.88/chromedriver-mac-arm64/chromedriver] found in cache
2025-03-18 18:35:36,659 - INFO - Scraping jobs from Amazon
2025-03-18 18:35:38,480 - INFO - Clicked popup/cookie button with xpath: //button[contains(@id, 'accept')]
